In [ ]:
!pip install tensorflow
!pip install tensorflow-gpu

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import layers
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# Constants
image_data_dir = r"C:\Users\siddh\Downloads\archive (2)\Data\images_original"
additional_data_file = r"C:\Users\siddh\Downloads\archive (2)\Data\features_30_sec.csv"
num_features = 58  # Number of features in the additional dataset

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# Get list of subdirectories (each representing a class)
class_directories = [os.path.join(image_data_dir, d) for d in os.listdir(image_data_dir) if os.path.isdir(os.path.join(image_data_dir, d))]

# Set the number of images to display per row
images_per_row = 5

# Create subplots with specified number of rows and columns
num_rows = len(class_directories)
num_cols = images_per_row
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 3*num_rows))

# Iterate over each class directory
for i, class_dir in enumerate(class_directories):
    # Get list of image files in the directory
    image_files = [f for f in os.listdir(class_dir) if f.endswith('.png') or f.endswith('.jpg')]
    if image_files:
        # Take the first 5 images from the directory (or less if there are fewer than 5 images)
        for j in range(min(len(image_files), images_per_row)):
            image_path = os.path.join(class_dir, image_files[j])
            # Load and display the image
            img = mpimg.imread(image_path)
            ax = axes[i, j] if num_rows > 1 else axes[j]  # Handle single row case
            ax.imshow(img)
            ax.axis('off')  # Turn off axis numbers and ticks
            ax.set_title(os.path.basename(class_dir))  # Set title as directory name

# Adjust layout and display the plot
plt.tight_layout()
plt.show()


In [ ]:
# Function to load and preprocess the image dataset
def load_image_dataset(image_data_dir):
    image_data = []
    labels = []
    for label in os.listdir(image_data_dir):
        label_dir = os.path.join(image_data_dir, label)
        if os.path.isdir(label_dir):
            for filename in os.listdir(label_dir):
                if filename.endswith('.png') or filename.endswith('.jpg'):
                    image_path = os.path.join(label_dir, filename)
                    img = mpimg.imread(image_path)
                    #img = img / 255.0
                    image_data.append(img)
                    labels.append(label)
    return image_data, labels

# Function to load and preprocess the additional dataset
def load_additional_dataset(additional_data_file):
    additional_data_df = pd.read_csv(additional_data_file)
    additional_data_df.drop(columns=["length"], inplace=True)
    scaler = MinMaxScaler()
    additional_data_df[additional_data_df.columns[1:-1]] = scaler.fit_transform(additional_data_df[additional_data_df.columns[1:-1]])
    return additional_data_df

# Load image dataset
image_data, labels = load_image_dataset(image_data_dir)

# Load additional dataset
additional_data_df = load_additional_dataset(additional_data_file)


print("Number of samples in image_data:", len(image_data))
print("Number of samples in additional_data_df:", len(additional_data_df))
print("Number of samples in Labels:", len(labels))

In [ ]:
print("Data Type of X_train_img:", image_data[0].dtype)
print("Shape of X_train_img:", image_data[0].shape)

In [ ]:
additional_data_df

In [ ]:
# Assuming you have a target label for each image, convert labels to one-hot encoding
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
one_hot_labels = to_categorical(encoded_labels)

# Define the columns to exclude
columns_to_exclude = ['filename', 'label']

# Select only the columns you need for training and validation
selected_columns = [col for col in additional_data_df.columns if col not in columns_to_exclude]

# Split the data
X_train_img, X_val_img, X_train_features, X_val_features, y_train, y_val = train_test_split(
    image_data,
    additional_data_df[selected_columns].values,  # Selecting only the required columns
    one_hot_labels,
    test_size=0.2,
    random_state=42
)

In [ ]:
# For numpy arrays
print("Size of X_train_features:", X_train_features.shape)
print("Size of X_val_features:", X_val_features.shape)
print("Size of y_train:", y_train.shape)
print("Size of y_val:", y_val.shape)

In [ ]:
# Define CNN model for image processing
img_height, img_width = 288, 432
# Define input shape
input_shape = (32, 288, 432, 4)
num_features = (None, 57)

# Define CNN model for image processing
image_input = tf.keras.Input(shape=input_shape[1:])
#image_input = tf.keras.Input(shape=(img_height, img_width, 4))
x = layers.Conv2D(64, (3, 3), activation='relu')(image_input)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(256, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(256, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(512, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Flatten()(x)
image_output = layers.Dense(256, activation='relu')(x)

# Define the input layer for the features dataset
features_input = tf.keras.Input(shape=num_features[1:])

# Concatenate the outputs of the CNN model and the features input
concatenated = layers.concatenate([image_output, features_input])

# Define additional dense layers for further processing
x = layers.Dense(128, activation='relu')(concatenated)
x = layers.Dense(64, activation='relu')(x)

# Output layer
output = layers.Dense(10, activation='softmax')(x)

# Define the final model
model = tf.keras.Model(inputs=[image_input, features_input], outputs=output)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
# Convert image arrays to tensors
X_train_img_tensor = tf.convert_to_tensor(X_train_img)
X_val_img_tensor = tf.convert_to_tensor(X_val_img)

print("Data Type of X_train_img:", X_train_img_tensor.dtype)
print("Shape of X_train_img:", X_train_img_tensor.shape)

print("Data Type of X_val_img:", X_val_img_tensor.dtype)
print("Shape of X_val_img:", X_val_img_tensor.shape)

In [ ]:
# Fit the model
history = model.fit([X_train_img_tensor, X_train_features], y_train,
                    validation_data=([X_val_img_tensor, X_val_features], y_val), 
                    epochs=30, 
                    batch_size=32)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Assuming you have already trained your model and obtained the 'history' object
# model.fit([...])

# Obtain predictions on validation data
y_pred_proba = model.predict([X_val_img_tensor, X_val_features])
y_pred = np.argmax(y_pred_proba, axis=1)

# Convert multilabel-indicator to single array of integers
y_val_labels = np.argmax(y_val, axis=1)

# Compute evaluation metrics
accuracy = accuracy_score(y_val_labels, y_pred)
precision = precision_score(y_val_labels, y_pred, average='weighted')
recall = recall_score(y_val_labels, y_pred, average='weighted')
f1 = f1_score(y_val_labels, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
input_shape1 = (32, 288, 432, 4)

# Define CNN model for image processing
image_input1 = tf.keras.Input(shape=input_shape1[1:])
#image_input = tf.keras.Input(shape=(img_height, img_width, 4))
x = layers.Conv2D(64, (3, 3), activation='relu')(image_input1)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(256, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(256, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(512, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
output1 = layers.Dense(10, activation='softmax')(x)

# Define the final model
model1 = tf.keras.Model(inputs=[image_input1], outputs=output1)

# Compile the model
model1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model1.summary()

In [ ]:
history1 = model1.fit([X_train_img_tensor], y_train,
                    validation_data=([X_val_img_tensor], y_val), 
                    epochs=30, 
                    batch_size=32)

In [ ]:
# Obtain predictions on validation data
y_pred_proba1 = model1.predict([X_val_img_tensor])
y_pred1 = np.argmax(y_pred_proba1, axis=1)

# Convert multilabel-indicator to single array of integers
y_val_labels1 = np.argmax(y_val, axis=1)

# Compute evaluation metrics
accuracy1 = accuracy_score(y_val_labels1, y_pred1)
precision1 = precision_score(y_val_labels1, y_pred1, average='weighted')
recall1 = recall_score(y_val_labels1, y_pred1, average='weighted')
f11 = f1_score(y_val_labels1, y_pred1, average='weighted')

print("Accuracy:", accuracy1)
print("Precision:", precision1)
print("Recall:", recall1)
print("F1 Score:", f11)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data
algorithms = ['Random Forest', 'XGB', 'LSTM', 'CNN']
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
rf_values = [0.8571428571428571, 0.8572001243506782, 0.8576325875124603, 0.8559601851531531,
             0.8878, 0.8874, 0.8881, 0.8870]
xgb_values = [0.8932362754018806, 0.8934990005949899, 0.8937220039659785, 0.8930229775741804,
              0.9029, 0.9028, 0.9033, 0.9028]
lstm_values = [0.9224, 0.9221, 0.922, 0.9217,
               0.9054, 0.9058, 0.9053, 0.9054]
cnn_values = [0.42, 0.4572360206339335, 0.42, 0.43180579228963095,
             0.66, 0.5614711701342135, 0.535, 0.5407217221732509]

# Plotting
fig, axes = plt.subplots(4, 1, figsize=(10, 20))

for i, metric in enumerate(metrics):
    ax = axes[i]
    bar_width = 0.35
    index = np.arange(len(algorithms))
    ax.bar(index - bar_width/2, [rf_values[i], xgb_values[i], lstm_values[i], cnn_values[i]], bar_width, label='Pre-Tuning')
    ax.bar(index + bar_width/2, [rf_values[i+4], xgb_values[i+4], lstm_values[i+4], cnn_values[i+4]], bar_width, label='Post-Tuning')
    ax.set_xlabel('Algorithms')
    ax.set_ylabel(metric)
    ax.set_xticks(index)
    ax.set_xticklabels(algorithms)
    ax.legend()

plt.tight_layout()
plt.show()



In [ ]:
accuracy1 = [0.1428, 0.2651, 0.3118, 0.3767, 0.3943, 0.4826, 0.5344, 0.5976, 0.5457, 0.6263, 0.6571, 0.6473, 0.6669, 0.7010, 0.7172, 0.7340, 0.7560, 0.7553, 0.8413, 0.8176]
loss1 = [2.3602, 2.2222, 2.1178, 1.9549, 1.7423, 1.5332, 1.3737, 1.2798, 1.2806, 1.1256, 1.0824, 1.0399, 0.9748, 0.8987, 0.8448, 0.7857, 0.7057, 0.6566, 0.5268, 0.5183]
val_accuracy1 = [0.1500, 0.2150, 0.3150, 0.3200, 0.4200, 0.5050, 0.5450, 0.4500, 0.5700, 0.5850, 0.5550, 0.6150, 0.6100, 0.6100, 0.6150, 0.6550, 0.6150, 0.6450, 0.5600, 0.6600]
val_loss1 = [2.2664, 2.1856, 2.0665, 1.8547, 1.6667, 1.4419, 1.3817, 1.4087, 1.2387, 1.2014, 1.1974, 1.1191, 1.1375, 1.0967, 1.0597, 1.0910, 1.0304, 1.1597, 1.2177, 1.0096]

accuracy2 = [0.0737, 0.1088, 0.1375, 0.1779, 0.2177, 0.2741, 0.2610, 0.3150, 0.3347, 0.3557, 0.3824, 0.4087, 0.4419, 0.4351, 0.4871, 0.5421, 0.5947, 0.6398, 0.6699, 0.7267]
loss2 = [2.3849, 2.3015, 2.2860, 2.1764, 2.0949, 1.9899, 1.9861, 1.8675, 1.8718, 1.7999, 1.7546, 1.6350, 1.6067, 1.5835, 1.4689, 1.2682, 1.1605, 1.0291, 0.9281, 0.8064]
val_accuracy2 = [0.0650, 0.0650, 0.1700, 0.2600, 0.2550, 0.2350, 0.2650, 0.3100, 0.2900, 0.2800, 0.3450, 0.3350, 0.3800, 0.3650, 0.3600, 0.3450, 0.3750, 0.4000, 0.4000, 0.4250]
val_loss2 = [2.3030, 2.3038, 2.2261, 2.0776, 1.9864, 1.9765, 1.9178, 1.8655, 1.8811, 1.9306, 1.8719, 1.8275, 1.8799, 1.8763, 1.9635, 1.9437, 2.0844, 2.2846, 2.1514, 2.1100]



In [ ]:
import matplotlib.pyplot as plt

# Data
epochs = list(range(1, 21))

# First set of data
accuracy1 = [0.1428, 0.2651, 0.3118, 0.3767, 0.3943, 0.4826, 0.5344, 0.5976, 0.5457, 0.6263, 0.6571, 0.6473, 0.6669, 0.7010, 0.7172, 0.7340, 0.7560, 0.7553, 0.8413, 0.8176]
loss1 = [2.3602, 2.2222, 2.1178, 1.9549, 1.7423, 1.5332, 1.3737, 1.2798, 1.2806, 1.1256, 1.0824, 1.0399, 0.9748, 0.8987, 0.8448, 0.7857, 0.7057, 0.6566, 0.5268, 0.5183]
val_accuracy1 = [0.1500, 0.2150, 0.3150, 0.3200, 0.4200, 0.5050, 0.5450, 0.4500, 0.5700, 0.5850, 0.5550, 0.6150, 0.6100, 0.6100, 0.6150, 0.6550, 0.6150, 0.6450, 0.5600, 0.6600]
val_loss1 = [2.2664, 2.1856, 2.0665, 1.8547, 1.6667, 1.4419, 1.3817, 1.4087, 1.2387, 1.2014, 1.1974, 1.1191, 1.1375, 1.0967, 1.0597, 1.0910, 1.0304, 1.1597, 1.2177, 1.0096]

# Second set of data
accuracy2 = [0.0737, 0.1088, 0.1375, 0.1779, 0.2177, 0.2741, 0.2610, 0.3150, 0.3347, 0.3557, 0.3824, 0.4087, 0.4419, 0.4351, 0.4871, 0.5421, 0.5947, 0.6398, 0.6699, 0.7267]
loss2 = [2.3849, 2.3015, 2.2860, 2.1764, 2.0949, 1.9899, 1.9861, 1.8675, 1.8718, 1.7999, 1.7546, 1.6350, 1.6067, 1.5835, 1.4689, 1.2682, 1.1605, 1.0291, 0.9281, 0.8064]
val_accuracy2 = [0.0650, 0.0650, 0.1700, 0.2600, 0.2550, 0.2350, 0.2650, 0.3100, 0.2900, 0.2800, 0.3450, 0.3350, 0.3800, 0.3650, 0.3600, 0.3450, 0.3750, 0.4000, 0.4000, 0.4250]
val_loss2 = [2.3030, 2.3038, 2.2261, 2.0776, 1.9864, 1.9765, 1.9178, 1.8655, 1.8811, 1.9306, 1.8719, 1.8275, 1.8799, 1.8763, 1.9635, 1.9437, 2.0844, 2.2846, 2.1514, 2.1100]

# Plotting
plt.figure(figsize=(10, 6))

plt.subplot(2, 1, 1)
plt.plot(epochs, accuracy1, label='Accuracy 1', marker='o')
plt.plot(epochs, accuracy2, label='Accuracy 2', marker='x')
plt.title('Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, loss1, label='Loss 1', marker='o')
plt.plot(epochs, loss2, label='Loss 2', marker='x')
plt.title('Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 1)
plt.plot(epochs, val_accuracy1, label='Validation Accuracy 1', marker='o')
plt.plot(epochs, val_accuracy2, label='Validation Accuracy 2', marker='x')
plt.title('Validation Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, val_loss1, label='Validation Loss 1', marker='o')
plt.plot(epochs, val_loss2, label='Validation Loss 2', marker='x')
plt.title('Validation Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Validation Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

plt.subplot(2, 1, 1)
plt.plot(epochs, val_accuracy1, label='Validation Accuracy 1', marker='o')
plt.plot(epochs, val_accuracy2, label='Validation Accuracy 2', marker='x')
plt.title('Validation Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, val_loss1, label='Validation Loss 1', marker='o')
plt.plot(epochs, val_loss2, label='Validation Loss 2', marker='x')
plt.title('Validation Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Validation Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Plotting
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(epochs, accuracy1, label='Images + Features Data')
plt.plot(epochs, accuracy2, label='Images')
plt.title('Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(epochs, loss1, label='Images + Features Data')
plt.plot(epochs, loss2, label='Images')
plt.title('Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(epochs, val_accuracy1, label='Images + Features Data')
plt.plot(epochs, val_accuracy2, label='Images')
plt.title('Validation Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(epochs, val_loss1, label='Images + Features Data')
plt.plot(epochs, val_loss2, label='Images')
plt.title('Validation Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Validation Loss')
plt.legend()

plt.tight_layout()
plt.show()